# Lambdas functions

Lambda (or anonymous) functions were introduced in C++11 and enhanced in C++14. They can be defined on-the-fly anywhere in the code where they are needed. This avoid a previous declaration of the function, useless when the function is to be used only once. Moreover, the capture of local variables avoids the tedious definition of an object-function (whose `operator()` is overloaded). In simple cases, the resulting code is more concise and clearer. If you write very complicated lambda functions, it is less obvious... it's up to you to judge.

## One-time use, anonymous, functions

Some ordinary functions are meant to be used only once. Since nested functions are not allowed in C++, one must pollute the global namespace:

In [1]:
#include <vector>

In [2]:
int reduce( std::vector<int> collection, int accumulator, int (*op)( int, int ) )
 {
  for ( int element : collection )
   { accumulator = op(accumulator,element) ; }
  return accumulator ;
 }

In [3]:
int add( int val1, int val2 ) { return (val1+val2) ; }

In [4]:
int multiply( int val1, int val2 ) { return (val1*val2) ; }

In [5]:
#include <iostream>

In [6]:
std::vector<int> numbers = { 1, 2, 3, 4, 5 } ;
std::cout<<reduce(numbers,0,add)<<std::endl ;
std::cout<<reduce(numbers,1,multiply)<<std::endl ;

15
120


C++11 allows one-the-fly definition of an anonymous function, where it is to be used. **The function name is replaced with `[]`**. It is called a *lambda*:

In [7]:
std::vector<int> numbers = { 1, 2, 3, 4, 5 } ;
std::cout<<reduce(numbers,0,[](int i1, int i2){ return i1+i2 ; })<<std::endl ;
std::cout<<reduce(numbers,1,[](int i1, int i2){ return i1*i2 ; })<<std::endl ;

15
120


Like any ordinary function, it only access its own input arguments, local variables and file global variables. Below, we try to raise our elements to a given exponent, but this will not compile as is:

In [8]:
#include <cmath>

In [9]:
std::vector<int> numbers = { 1, 2, 3, 4, 5 } ;
int exponent = 2 ;
int res = reduce(numbers,0,[]( int i1, int i2 )
 { return i1+std::pow(i2,exponent) ; }) ;

input_line_15:5:26: error: variable 'exponent' cannot be implicitly captured in a lambda with no capture-default specified
 { return i1+std::pow(i2,exponent) ; }) ;
                         ^
input_line_15:3:5: note: 'exponent' declared here
int exponent = 2 ;
    ^
input_line_15:4:28: note: lambda expression begins here
int res = reduce(numbers,0,[]( int i1, int i2 )
                           ^
input_line_15:4:11: error: no matching function for call to 'reduce'
int res = reduce(numbers,0,[]( int i1, int i2 )
          ^~~~~~
input_line_8:1:5: note: candidate function not viable: no known conversion from '(lambda at input_line_15:4:28)' to 'int (*)(int, int)' for 3rd argument
int reduce( std::vector<int> collection, int accumulator, int (*op)( int, int ) )
    ^
/opt/conda/bin/../lib/gcc/../../x86_64-conda-linux-gnu/include/c++/9.4.0/numeric:296:5: note: candidate template ignored: deduced conflicting types for parameter '_InputIterator' ('std::vector<int, std::allocator<int> >' vs. 

Interpreter Error: 

A classical way to solve the preceding issue would be to generalize the `reduce()` function so to be able to handle any kind of "callable" as operator, and write an object-function:

In [10]:
#include <functional>

In [11]:
int reduce( std::vector<int> collection, int accumulator, std::function<int(int,int)> op )
 {
  for ( int element : collection )
   { accumulator = op(accumulator,element) ; }
  return accumulator ;
 }

In [12]:
class AddPower
 {
  public :
    AddPower( int exponent ) : m_exponent{exponent} {}
    int operator()( int i1, int i2 ) const
     { return i1+std::pow(i2,m_exponent) ; }
  private :
    int m_exponent ;
 } ;

In [13]:
std::vector<int> numbers = { 1, 2, 3, 4, 5 } ;
AddPower add_power{2} ;
std::cout<<reduce(numbers,0,add_power)<<std::endl ;

55


## Local variables capture

Instead, a lambda can include, between its square brackets (`[]`), a list of context variables to be captured, by value or by reference:
* `[x,y,&j]` : `x` and `y` by value, `j` by reference ;
* `[]` : nothing captured ;
* `[&]` : all variables by reference ;
* `[=]` : all variables by value ;
* `[=,&j]` : all variables by value, except `j` by reference ;
* `[&,j]` : all variables by reference, except `j` by value.

That's where the lambdas become a killing feature. The previous example, which were requiring the object-function `AddPower`, can be greatly simplified with a lambda and a capture:

In [14]:
std::vector<int> numbers = { 1, 2, 3, 4, 5 } ;
int exponent = 2 ;

int res = reduce(numbers,0,
 [exponent]( int i1, int i2 )
 { return i1+std::pow(i2,exponent) ; }) ;

std::cout << res << std::endl ;

55


At last, take care that if you want to use a member of the current object, you must explicitly capture `this`: 

In [15]:
class Process
 {
  public :
    Process( int exponent ) : m_exponent{exponent} {}
    void run()
     {
      std::vector<int> numbers = { 1, 2, 3, 4, 5 } ;
      int res = reduce(numbers,0,
       [this]( int i1, int i2 )
       { return i1+std::pow(i2,m_exponent) ; }) ;
      std::cout<<res<<std::endl ;
     }
  private :
    int m_exponent ;
 } ; 

In [16]:
Process p{2} ;
p.run() ;

55


## Return type

The compiler can infer the return type of your lambdas, but sometimes you may want to declare it explicitly, especially when the compiler guess may be wrong. In the example below, we explicitly declare that the lambda return an `int`:

In [1]:
// RESTART KERNEL

#include <vector>
#include <iostream>
#include <functional>

int reduce( std::vector<int> collection, int accumulator, std::function<int(int,int)> op )
 {
  for ( int element : collection )
   { accumulator = op(accumulator,element) ; }
  return accumulator ;
 }

In [2]:
std::vector<int> numbers = { 1, 2, 3, 4, 5 } ;
int res = reduce(numbers,0,
 [] (int i1, int i2) -> int
 { return i1+i2 ; }) ;
std::cout << res << std::endl ;

15


## Storing and reusing a lambda

As you understood, a lambda is a kind of object-function. As such, it can be stored in a variable and reused. The type of such a lambda is somehow tricky, and the use of `auto` is the only way to declare a variable containing a lambda:

***FEATURE BELOW NOT SUPPORTED BY THE C++17 KERNEL***

In [3]:
auto display = []( int i ) -> void
{ std::cout << i << std::endl ; } ;

In [ ]:
for ( int i : { 1, 2, 3,4, 5 } )
 { display(i) ; }

In such trivial examples, a lambda has no visible benefit over an ordinary function. Yet, one can notice that **you can nest your `display` function definition** where you want, although an ordinary function cannot be nested in C++.

## Generic lambdas

Starting with C++14, one can replace the types of a lambda arguments with `auto`.

In [3]:
#include <algorithm>

In [5]:
auto display = []( auto v ) -> void
{ std::cout << v << " " ; } ;
     
std::vector<int> numbers1 = { 1, 2, 3 } ;
std::vector<double> numbers2 = { 4.4, 5.5, 6.6 } ;
  
std::for_each(numbers1.begin(),numbers1.end(),display) ;
std::for_each(numbers2.begin(),numbers2.end(),display) ;

1 2 3 4.4 5.5 6.6 

This `display` lambda **should not** be considered as templated function. It is rather equivalent to an object-function with a template `operator()` method. Something like this :

In [ ]:
class Display
 {
  public :
    template <typename Value>
    void operator()( Value v )
     { std::cout << v << " " ; }
 } ;

Note : the use of `auto` in lambdas has probably inspired the "abbreviated function template" in C++20.

## Generating lambdas

Last but not least, note that you can now write functions which produce functions as a result:

In [1]:
#include <iostream>

In [2]:
auto make_multiplier( int m )
 { return [m]( int n ) -> void { std::cout << (m*n) << " " ; } ; }

In [3]:
#include <vector>
#include <algorithm>

In [4]:
{
  std::vector<int> numbers = { 1, 2, 3, 4, 5 } ;
  auto mult = make_multiplier(3) ;
  std::for_each(numbers.begin(),numbers.end(),mult) ;
}

3 6 9 12 15 

But in such a case, beware not to capture by reference some local variable:

In [9]:
auto make_multiplier_2( int m )
 { return [&]( int n ) -> void { std::cout << (m*n) << " " ; } ; }

In [10]:
std::vector<int> numbers = { 1, 2, 3, 4, 5 } ;
auto mult = make_multiplier_2(3) ;
std::for_each(numbers.begin(),numbers.end(),mult) ;

32764 65528 98292 131056 163820 

## Exercise

Below, replace `random_complex` and `ComplexPow` with lambdas.

In [ ]:
%%file tmp.lambda-functions.cpp

#include <complex>
#include <vector>
#include <algorithm>
#include <iostream>
#include <cmath>

using Real = double ;
using Complex = std::complex<Real> ;
using Complexes = std::vector<Complex> ;

// random complex on the unit circle
void random_complex( Complex & c )
 { 
  Real d = ((Real)rand())/RAND_MAX ;
  Real e = 2*M_PI*d ;
  c = Complex(cos(e),sin(e)) ;
 }

// print the global product of n complex numbers 
void print_product( Complexes const & cs )
 {
  Complex z(1.,0.) ;
  for ( auto c : cs )
   { z *= c ; }
  std::cout<<z<<"\n" ;
 }

// compute x^d
struct ComplexPow
 {
  int m_degree ;
  ComplexPow( int degree ) : m_degree(degree) {}
  Complex operator()( Complex const & x ) const
   {
    int d ;
    Complex y(1.,0.) ;
    for ( d=0 ; d<m_degree ; ++d )
     { y *= x ; }
    return y ;
   }
 } ;

// programme principal
int main ( int argc, char * argv[] )
 {
  int dim = atoi(argv[1]) ;
  int degree = atoi(argv[2]) ;
  
  // generate random input
  Complexes input(dim) ;
  srand(201911104) ;
  std::for_each(input.begin(),input.end(),random_complex) ;

  // compute output
  Complexes output(dim) ;
  std::transform(input.begin(),input.end(),output.begin(),ComplexPow(degree)) ;
  
  // check result
  std::cout.precision(2) ;
  print_product(output) ;

  return 0 ;
 }

In [ ]:
!rm -f tmp.lambda-functions.exe && g++ -std=c++17 tmp.lambda-functions.cpp -o tmp.lambda-functions.exe

In [ ]:
!./tmp.lambda-functions.exe 5 10

© *CNRS 2020*  
*Assembled and written by David Chamont, this work is made available according to the terms of the*  
[*Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International*](http://creativecommons.org/licenses/by-nc-sa/4.0/)